Train substitute model and attack test data

Dataset:
```
|-----------------------------------|-valid-|-test--|
|------target-----|----substitute---|-valid-|-test--|
| train agg_model | train rnn_model |       |       |
|                 | make adversarial for v and  t
| estimate score on original and adv v and t
```

In [1]:
%cd ../

/mnt2/kireev/pycharm-deploy/pytorch-lifestream/experiments/scenario_gender


In [2]:
import sys
sys.path.append('../../')

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [4]:
import logging

In [5]:
from itertools import chain, islice, product, cycle

In [6]:
import json
import jsonlines
import matplotlib.pyplot as plt
import lightgbm
import numpy as np
import pandas as pd
import pickle
import pytorch_lightning as pl
import random
import torch

In [7]:
from copy import deepcopy
from tqdm.auto import tqdm
from pyhocon import ConfigFactory
from torch.utils.data import DataLoader
from sklearn.decomposition import PCA
from IPython.display import display

In [8]:
from sklearn.metrics import roc_auc_score, recall_score, precision_score

In [9]:
from embeddings_validation.tasks.report_collect import t_pm, t_int_l, t_int_h

In [10]:
from dltranz.data_load import padded_collate, IterableChain, IterableAugmentations
from dltranz.data_load.augmentations.build_augmentations import build_augmentations
from dltranz.data_load.data_module.map_augmentation_dataset import MapAugmentationDataset
from dltranz.data_load.iterable_processing.category_size_clip import CategorySizeClip
from dltranz.data_load.iterable_processing.feature_filter import FeatureFilter
from dltranz.data_load.iterable_processing.feature_type_cast import FeatureTypeCast
from dltranz.data_load.iterable_processing.seq_len_filter import SeqLenFilter
from dltranz.data_load.parquet_dataset import ParquetFiles, ParquetDataset
from dltranz.data_load.list_splitter import ListSplitter
from dltranz.data_load.iterable_processing.target_join import TargetJoin
from dltranz.data_load.iterable_processing.id_filter import IdFilter
from dltranz.data_load.iterable_processing_dataset import IterableProcessingDataset
from dltranz.models import create_head_layers
from dltranz.seq_encoder import create_encoder
from dltranz.seq_to_target import SequenceToTarget
from dltranz.train import score_model2
from dltranz.trx_encoder import PaddedBatch
from dltranz.util import get_cls

In [11]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)-7s %(funcName)-20s   : %(message)s')

In [12]:
random.randint(1, 2**16)

57034

In [13]:
config_text = """
{
  seed_everything: 42

  data_module: {
    setup: {
      dataset_files: {
        train_data_path: "data/train_trx.parquet"
        test_data_path: "data/test_trx.parquet"
      }
      target_file_path: "data/gender_train.csv"
      
      col_id: customer_id
      col_id_dtype: int
      col_target: gender

      split_seed: 23921
      cv_folds: 5
      substitute_part: 0.5
      valid_part: 0.1
    }
    train: {
        min_seq_len: 0
        augmentations: [
            [RandomSlice, {min_len: 250, max_len: 350, rate_for_min: 0.9}]
            [DropoutTrx, {trx_dropout: 0.01}]
        ]
        num_workers: 16
        batch_size: 32
        drop_last: true
    }

    valid: {
        augmentations: [
            [SeqLenLimit, {max_seq_len: 1200}]
        ]
        num_workers: 8
        batch_size: 64   # 512
        drop_last: true
    }
    test: {
        augmentations: [
            [SeqLenLimit, {max_seq_len: 1200}]
        ]
        num_workers: 0
        batch_size: 500
    }
  }
  
  substitute:
  {
      logger_name: adv_substitute
      trainer: {
        gpus: 1
        auto_select_gpus: false

        # max_epochs: 50
        max_epochs: 50
        
        checkpoint_callback: false
        deterministic: True
      }
      params: {
        score_metric: [auroc, accuracy]

        encoder_type: rnn,
        trx_encoder: {
          norm_embeddings: false,
          embeddings_noise: 0.003,
          embeddings: {
            mcc_code: {in: 200, out: 48},
            tr_type: {in: 100, out: 24}
          },
          numeric_values: {
            amount: identity
          }
        },
        rnn: {
          hidden_size: 48,
          type: gru,
          bidir: false,
          trainable_starter: static
        },

        head_layers: [
            [BatchNorm1d, {num_features: "{seq_encoder.embedding_size}"}]
            [Linear, {"in_features": "{seq_encoder.embedding_size}", "out_features": 1}]
            [Sigmoid, {}]
            [Squeeze, {}]
        ]

        train: {
          random_neg: false,
          loss: bce,
          lr: 0.005,
          weight_decay: 0.0,
        },
        lr_scheduler: {
          step_size: 1,
          step_gamma: 0.70
        }
      }
  }
  
  attack: {
      mcc_len: 2
      max_mcc: 70
      # sample_rate: 0.1
      sample_rate: 0.1
  }
  
  agg_target: {
    params: {
        encoder_type: agg_features
        trx_encoder: {
          embeddings: {
            mcc_code: {in: 200}
            tr_type: {in: 100}
          }
          numeric_values: {
            amount: identity
          }
          was_logified: true
          log_scale_factor: 1
        }
    }
  }
  coles_target: {
        trainer: {
            # max_epochs: 150
            # max_epochs: 5
            max_epochs: 100
        }
        train: {
          min_seq_len: 25
          split_strategy: {
            split_strategy: "SampleSlices"
            split_count: 5
            cnt_min: 15
            cnt_max: 75
          }
          trx_dropout: 0.01
          num_workers: 8
          batch_size: 128
        }
        valid: {
          split_strategy: {
            split_strategy: SampleSlices
            split_count: 5
            cnt_min: 25
            cnt_max: 100
          }
          num_workers: 16
          batch_size: 1024
        }
      params: {
            validation_metric_params: {
                K: 4
                metric: cosine
            }

            encoder_type: rnn,
            trx_encoder: {
              norm_embeddings: false,
              embeddings_noise: 0.003,
              embeddings: {
                mcc_code: {in: 200, out: 48},
                tr_type: {in: 100, out: 24}
              },
              numeric_values: {
                amount: identity
              }
            },
            rnn: {
              type: gru,
              hidden_size: 512,
              bidir: false,
              trainable_starter: static
            },
            head_layers: [
                [NormEncoder, {}],
            ]

            lr_scheduler: {
              step_size: 10,
              step_gamma: 0.9025
            },
            train: {
              sampling_strategy: HardNegativePair,
              neg_count: 5,
              loss: ContrastiveLoss,
              margin: 0.5,
              lr: 0.002,
              weight_decay: 0.0
            }
      }
  }
  target_rnn: {
      trainer: {
        gpus: 1
        auto_select_gpus: false

        # max_epochs: 50
        max_epochs: 25
        
        deterministic: True
      }
      params: {
        score_metric: [auroc, accuracy]

        encoder_type: rnn,
        trx_encoder: {
          norm_embeddings: false,
          embeddings_noise: 0.003,
          embeddings: {
            mcc_code: {in: 200, out: 48},
            tr_type: {in: 100, out: 24}
          },
          numeric_values: {
            amount: identity
          }
        },
        rnn: {
          hidden_size: 48,
          type: gru,
          bidir: false,
          trainable_starter: static
        },

        head_layers: [
            [BatchNorm1d, {num_features: "{seq_encoder.embedding_size}"}]
            [Linear, {"in_features": "{seq_encoder.embedding_size}", "out_features": 1}]
            [Sigmoid, {}]
            [Squeeze, {}]
        ]

        train: {
          random_neg: false,
          loss: bce,
          lr: 0.005,
          weight_decay: 0.0,
        },
        lr_scheduler: {
          step_size: 1,
          step_gamma: 0.70
        }
      }
  }
  lgbm_params: {
        n_estimators: 500
        boosting_type: gbdt
        objective: binary
        metric: auc
        subsample: 0.5
        subsample_freq: 1
        learning_rate: 0.02
        feature_fraction: 0.75
        max_depth: 6
        lambda_l1: 1
        lambda_l2: 1
        min_data_in_leaf: 50
        random_state: 42
        n_jobs: 8
        reg_alpha: null
        reg_lambda: null
        colsample_bytree: null
        min_child_samples: null
  }

}
"""

conf = ConfigFactory.parse_string(config_text)

# Classes

In [14]:
class TargetToDict(IterableProcessingDataset):
    def __init__(self, col_id):
        super().__init__()
        
        self.col_id = col_id
    
    def __iter__(self):
        for x, y in self._src:
            _id = x[self.col_id]
            yield x, {'id': _id, 'y': y}

In [15]:
from sklearn.model_selection import KFold

In [16]:
class DataSplitter:
    def __init__(self, conf):
        self.setup_conf = conf['setup']
        self.split()
        
    def split(self):
        train_data_files = ParquetFiles(self.setup_conf['dataset_files.train_data_path']).data_files
        test_data_files = ParquetFiles(self.setup_conf['dataset_files.test_data_path']).data_files

        train_data_files = np.array(train_data_files)
        kf = KFold(n_splits=self.setup_conf['cv_folds'], shuffle=True, random_state=self.setup_conf['split_seed']) 
        
        self.folds = {}
        for i, (i_train, i_valid) in enumerate(kf.split(train_data_files)):
            target_subst_spliter = ListSplitter(
                train_data_files[i_train].tolist(), valid_size=self.setup_conf['substitute_part'], 
                seed=self.setup_conf['split_seed'])
            substitute_splitter = ListSplitter(
                target_subst_spliter.valid, valid_size=self.setup_conf['valid_part'], 
                seed=self.setup_conf['split_seed'])

            self.folds[i] = {
                'test': test_data_files,
                'valid': train_data_files[i_valid].tolist(),
                'target': target_subst_spliter.train,
                'substitute_train': substitute_splitter.train,
                'substitute_valid': substitute_splitter.valid,
            }

In [17]:
class AdversarialTrainDataModule(pl.LightningDataModule):
    def __init__(self, conf, pl_module, *files):
        super().__init__()

        self.mode = None
        
        self.setup_conf = conf['setup']
        self.train_conf = conf['train']
        self.valid_conf = conf['valid']
        
        self.train_files, self.valid_files = files

        self.col_id = self.setup_conf['col_id']
        self.col_id_dtype = {
            'str': str,
            'int': int,
        }[self.setup_conf['col_id_dtype']]
        self.col_target = self.setup_conf['col_target']
        self.y_cast = int
        self.category_names = pl_module.seq_encoder.category_names
        self.category_names.add('event_time')
        self.category_max_size = pl_module.seq_encoder.category_max_size

    def prepare_data(self):
        self.load_target()
        self.setup_iterable_files()
        self.setup_map()

    def load_target(self):
        df = pd.read_csv(self.setup_conf['target_file_path'])
        df[self.col_id] = df[self.col_id].astype(self.col_id_dtype)
        df[self.col_target] = df[self.col_target].astype(self.y_cast)
        self._targets = df.set_index(self.col_id)[self.col_target].to_dict()
        
    def setup_iterable_files(self):
        self.train_dataset = ParquetDataset(
            self.train_files,
            post_processing=IterableChain(*self.build_iterable_processing('train')),
            shuffle_files=True,
        )
        self.valid_dataset = ParquetDataset(
            self.valid_files,
            post_processing=IterableChain(*self.build_iterable_processing('valid')),
            shuffle_files=False,
        )

    def build_iterable_processing(self, part):
        yield FeatureTypeCast({self.col_id: self.col_id_dtype})
        yield IdFilter(id_col=self.col_id, relevant_ids=list(self._targets.keys()))
        if part == 'train':
            yield SeqLenFilter(min_seq_len=self.train_conf['min_seq_len'])
        
        yield TargetJoin(self.col_id, self._targets, self.y_cast)

        yield FeatureFilter(keep_feature_names=self.category_names)
        yield CategorySizeClip(self.category_max_size)

    def build_augmentations(self, part):
        if part == 'train':
            return build_augmentations(self.train_conf['augmentations'])
        elif part == 'valid':
            return build_augmentations(self.valid_conf['augmentations'])

    def setup_map(self):
        self.train_dataset = list(tqdm(iter(self.train_dataset)))
        logger.info(f'Loaded {len(self.train_dataset)} for train model')
        self.valid_dataset = list(tqdm(iter(self.valid_dataset)))
        logger.info(f'Loaded {len(self.valid_dataset)} for valid model')

        self.train_dataset = MapAugmentationDataset(
            base_dataset=self.train_dataset,
            a_chain=self.build_augmentations('train'),
        )
        self.valid_dataset = MapAugmentationDataset(
            base_dataset=self.valid_dataset,
            a_chain=self.build_augmentations('valid'),
        )

    def train_dataloader(self):
        return DataLoader(
            dataset=self.train_dataset,
            collate_fn=padded_collate,
            shuffle=True,
            num_workers=self.train_conf['num_workers'],
            batch_size=self.train_conf['batch_size'],
            drop_last=self.train_conf.get('drop_last', False)
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=self.valid_dataset,
            collate_fn=padded_collate,
            num_workers=self.valid_conf['num_workers'],
            batch_size=self.valid_conf['batch_size'],
            drop_last=self.train_conf.get('drop_last', False),
        )

In [18]:
class GreadyAttacker:
    def __init__(self, is_sample_mcc=True, mcc_freq=None):
        self._src = None
        
        self.mcc_len = conf['attack.mcc_len']
        self.max_mcc = conf['attack.max_mcc']
        self.sample_rate = conf['attack.sample_rate']

        self.tr_for_mcc = {}
        for m in range(2, self.max_mcc):
            t = mcc_freq.loc[m]['tr_type_freq']
            t = t.index[torch.multinomial(
                torch.from_numpy(t.values), 1000, replacement=True).numpy()].values.tolist()
            self.tr_for_mcc[m] = cycle(t)
            
        self.amount_dist = mcc_freq[['a_min', 'a_max']].to_dict(orient='index')
        self.is_sample_mcc = is_sample_mcc

    def __call__(self, src):
        self._src = src
        return iter(self)

    def __iter__(self):
        for rec in self._src:
            for t in self.process(rec):
                yield t

    def process(self, x):
        # original record
        rec, d = x
        # yield rec, {'id': d['id'], 'y': d['y'], 'new_trx': None, 'new_trx_len': 0}
        
        if not self.is_sample_mcc:
            return 
        
        mcc_len = self.mcc_len
        mcc_product = list(product(*[range(2, self.max_mcc) for _ in range(mcc_len)]))
        n = int(len(mcc_product) * self.sample_rate)
        if n <= 1:
            print(n)
        sel_ids = set(np.random.choice(len(mcc_product), n, replace=False).tolist())
        for i, new_mcc in enumerate(mcc_product):
            if i not in sel_ids:
                continue
                
            new_rec = {}
            new_rec['mcc_code'] = torch.cat([rec['mcc_code'], torch.tensor(new_mcc)])
            
            v = rec['event_time'][-1]
            new_event_time = [v + 1, v + 2]
            new_rec['event_time'] = torch.cat([rec['event_time'], torch.tensor(new_event_time)])
            
            new_tr_type = []
            new_amount = []
            for m in new_mcc:
                t = next(self.tr_for_mcc[m])
                new_tr_type.append(t)
                
                ix = (rec['mcc_code'] == m) & (rec['tr_type'] == t)
                if ix.sum() == 0:
                    # no such trx for this client, sample similar trx
                    v = self.amount_dist[(m, t)]
                    a = random.random() * (v['a_max'] - v['a_min']) + v['a_min']
                else:
                    # take amount from history
                    v = rec['amount'][ix]
                    a = v[random.choice(range(len(v)))].item()         
                new_amount.append(a)
                
            new_rec['tr_type'] = torch.cat([rec['tr_type'], torch.tensor(new_tr_type)])
            new_rec['amount'] = torch.cat([rec['amount'], torch.tensor(new_amount)])
                
            yield new_rec, {'id': d['id'], 'y': d['y'], 'new_trx': {
                'mcc_code': list(new_mcc),
                'tr_type': new_tr_type,
                'amount': new_amount,
                'event_time': new_event_time,
            }, 'new_trx_len': len(new_mcc)}

In [19]:
class AdversarialAttackDataModule(pl.LightningDataModule):
    def __init__(self, conf, pl_module, mcc_freq, *files):
        super().__init__()

        self.mode = None
        
        self.setup_conf = conf['setup']
        self.test_conf = conf.get('test')
        
        self.test_files = files

        self.col_id = self.setup_conf['col_id']
        self.col_id_dtype = {
            'str': str,
            'int': int,
        }[self.setup_conf['col_id_dtype']]
        self.col_target = self.setup_conf['col_target']
        self.y_cast = int
        self.category_names = pl_module.seq_encoder.category_names
        self.category_names.add('event_time')
        self.category_max_size = pl_module.seq_encoder.category_max_size

    def prepare_data(self):
        self.load_target()
        self.setup_iterable_files()

    def load_target(self):
        df = pd.read_csv(self.setup_conf['target_file_path'])
        df[self.col_id] = df[self.col_id].astype(self.col_id_dtype)
        df[self.col_target] = df[self.col_target].astype(self.y_cast)
        self._targets = df.set_index(self.col_id)[self.col_target].to_dict()
        
    def setup_iterable_files(self):
        self.test_dataset = ParquetDataset(
            self.test_files,
            post_processing=IterableChain(*self.build_iterable_processing('test')),
            shuffle_files=False,
        )
        self.history_dataset = ParquetDataset(
            self.test_files,
            post_processing=IterableChain(*self.build_iterable_processing('history')),
            shuffle_files=False,
        )

    def build_iterable_processing(self, part):
        yield FeatureTypeCast({self.col_id: self.col_id_dtype})
        yield IdFilter(id_col=self.col_id, relevant_ids=list(self._targets.keys()))
        
        yield TargetJoin(self.col_id, self._targets, self.y_cast)
        yield TargetToDict(self.col_id)
        yield FeatureFilter(keep_feature_names=self.category_names)
        yield CategorySizeClip(self.category_max_size)
        yield IterableAugmentations(self.build_augmentations(part))
        
        if part == 'test':
            yield GreadyAttacker(is_sample_mcc=True, mcc_freq=mcc_freq)
        else:
            yield GreadyAttacker(is_sample_mcc=False, mcc_freq=mcc_freq)

    def build_augmentations(self, part):
        return build_augmentations(self.test_conf['augmentations'])

    def test_dataloader(self):
        return DataLoader(
            dataset=self.test_dataset,
            collate_fn=padded_collate,
            shuffle=False,
            num_workers=self.test_conf['num_workers'],
            batch_size=self.test_conf['batch_size'],
            drop_last=False,
        )

In [20]:
def get_dataset(conf, seq_encoder, files, apply_attacks=[]):
    # load_target
    col_id = conf['setup.col_id']
    col_id_dtype = {
        'str': str,
        'int': int,
    }[conf['setup.col_id_dtype']]
    col_target = conf['setup.col_target']
    y_cast = int
    
    df = pd.read_csv(conf['setup.target_file_path'])
    df[col_id] = df[col_id].astype(col_id_dtype)
    df[col_target] = df[col_target].astype(y_cast)
    _targets = df.set_index(col_id)[col_target].to_dict()
    
    category_names = seq_encoder.category_names
    category_names.add('event_time')

    dataset = ParquetDataset(
        files,
        post_processing=IterableChain(
            FeatureTypeCast({col_id: col_id_dtype}),
            IdFilter(id_col=col_id, relevant_ids=list(_targets.keys())),
            TargetJoin(col_id, _targets, y_cast),
            TargetToDict(col_id),
            *apply_attacks,
            FeatureFilter(keep_feature_names=category_names),
            CategorySizeClip(seq_encoder.category_max_size),
            IterableAugmentations(build_augmentations(conf['test.augmentations'])),
        ),
        shuffle_files=False,
    )

    return dataset

In [21]:
class PretrainedAttack(IterableProcessingDataset):
    def __init__(self, col_id, attacks):
        super().__init__()
        
        self.col_id = col_id
        self.attacks = attacks
            
    def process(self, x):
        _id = x[self.col_id]
        a = self.attacks[_id]
        for k, v in a.items():
            x[k] = torch.cat([x[k], torch.tensor(v)])
        return x

# Run

In [22]:
if 'seed_everything' in conf:
    pl.seed_everything(conf['seed_everything'])

Global seed set to 42


In [23]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [24]:
data_splitter = DataSplitter(conf['data_module'])

In [25]:
def get_mcc_freq(substitute_dm):
    df_mcc_freq = [[], [], []]

    for rec in chain(
        substitute_dm.train_dataloader().dataset,
        substitute_dm.val_dataloader().dataset,
    ):
        for i, k in enumerate(['mcc_code', 'tr_type', 'amount']):
            df_mcc_freq[i].append(rec[0][k].numpy())

    df_mcc_freq = pd.DataFrame({
        'mcc_code': np.concatenate(df_mcc_freq[0]),
        'tr_type': np.concatenate(df_mcc_freq[1]), 
        'amount': np.concatenate(df_mcc_freq[2]),
    })

    df_mcc_freq1 = df_mcc_freq
    df_mcc_freq = df_mcc_freq.groupby('mcc_code')
    df_mcc_freq = pd.concat([
        df_mcc_freq.apply(lambda x: x['tr_type'].value_counts(normalize=True).rename('tr_type_freq'))
    ], axis=1)

    df_mcc_freq = pd.concat([
        df_mcc_freq,
        df_mcc_freq1.groupby(['mcc_code', 'tr_type']).apply(lambda x: x['amount'].quantile([0.25, 0.75])) 
        .rename(columns={0.25: 'a_min', 0.75: 'a_max'}),
    ], axis=1)
    return df_mcc_freq

In [26]:
def train_substitute_model(fold_id, fold, l_conf):
    _substitute_trainer_params = l_conf['trainer']
    if 'logger_name' in l_conf:
        _substitute_trainer_params['logger'] = pl.loggers.TensorBoardLogger(
            save_dir='lightning_logs',
            name=f'[{fold_id}]: ' + l_conf.get('logger_name'),
        )
    _substitute_trainer_params['callbacks'] = [
        pl.callbacks.EarlyStopping(monitor='val_auroc', mode='max', min_delta=0.0005, patience=2)
    ]
    substitute_trainer = pl.Trainer(**_substitute_trainer_params)

    substitute_model = SequenceToTarget(l_conf['params'])
    substitute_dm = AdversarialTrainDataModule(conf['data_module'], substitute_model,
                                               fold['substitute_train'], fold['substitute_valid'])
    substitute_dm.prepare_data()
    mcc_freq = get_mcc_freq(substitute_dm)

    
    substitute_trainer.fit(substitute_model, substitute_dm)
    substitute_valid_metrics = {
        name: float(mf.compute().item()) for name, mf in substitute_model.valid_metrics.items()}
    logger.info(f'[{fold_id}]: ' + ', '.join([f'valid_{name}: {v:.4f}'
                                              for name, v in substitute_valid_metrics.items()]))
    
    
    return substitute_model, mcc_freq

In [27]:
def estimate_data(embedding_model, cls_model, files, pretrained_attacks):
    # orig
    dataloader = DataLoader(
        dataset=get_dataset(conf['data_module'], embedding_model, files),
        collate_fn=padded_collate,
        shuffle=False,
        num_workers=conf['data_module.test.num_workers'],
        batch_size=conf['data_module.test.batch_size'],
        drop_last=False,
    )
    X, y = score_model2(embedding_model, dataloader, {'device': 'cuda'})
    y = np.array([e['y'] for e in y])
    orig_scores = cls_model.predict_proba(X)[:, 1]
    yield 'auroc_orig', roc_auc_score(y, orig_scores)
    yield 'recall_orig', recall_score(y, orig_scores > 0.5)
    yield 'precision_orig', precision_score(y, orig_scores > 0.5)

    p_a = [PretrainedAttack(conf['data_module.setup.col_id'], pretrained_attacks)]
    dataloader = DataLoader(
        dataset=get_dataset(conf['data_module'], embedding_model, files, p_a),
        collate_fn=padded_collate,
        shuffle=False,
        num_workers=conf['data_module.test.num_workers'],
        batch_size=conf['data_module.test.batch_size'],
        drop_last=False,
    )
    X, y = score_model2(embedding_model, dataloader, {'device': 'cuda'})
    y = np.array([e['y'] for e in y])
    att_scores = cls_model.predict_proba(X)[:, 1]
    yield 'auroc_adv',  roc_auc_score(y, att_scores)
    yield 'recall_adv', recall_score(y, att_scores > 0.5)
    yield 'precision_adv', precision_score(y, att_scores > 0.5)

    yield 'p_drop', (orig_scores - att_scores).mean()

In [28]:
def train_target_model_agg_feature(fold_id, fold, pretrained_attacks):        
    target_model = create_encoder(conf['agg_target.params'], is_reduce_sequence=True)
    train_dataloader = DataLoader(
        dataset=get_dataset(conf['data_module'], target_model, fold['target']),
        collate_fn=padded_collate,
        shuffle=False,
        num_workers=conf['data_module.test.num_workers'],
        batch_size=conf['data_module.test.batch_size'],
        drop_last=False,
    )
    train_X, train_y = score_model2(target_model, train_dataloader, {'device': 'cuda'})
    train_y = np.array([e['y'] for e in train_y])
    lgb_cls = lightgbm.LGBMClassifier(**conf['lgbm_params'])
    lgb_cls.fit(train_X, train_y)
    
    for fields in estimate_data(target_model, lgb_cls, files=fold['valid'],
                                pretrained_attacks=pretrained_attacks):
        yield 'agg_features', 'lgb_cls', 'valid', *fields
    for fields in estimate_data(target_model, lgb_cls, files=fold['test'],
                                pretrained_attacks=pretrained_attacks):
        yield 'agg_features', 'lgb_cls', 'test', *fields

In [29]:
from dltranz.lightning_modules.coles_module import CoLESModule
from dltranz.data_load.data_module.coles_data_module import coles_collate_fn
from dltranz.metric_learn.dataset import split_strategy
from dltranz.metric_learn.dataset.splitting_dataset import IterableSplittingDataset, MapSplittingDataset

In [30]:
def train_target_model_coles(fold_id, fold, pretrained_attacks):        
    target_module = CoLESModule(conf['coles_target.params'])
    train_files, valid_files = fold['target'][:-5], fold['target'][-5:]    
    
    category_names = target_module.seq_encoder.category_names
    category_names.add('event_time')
    train_dataset = ParquetDataset(
        train_files,
        post_processing=IterableChain(
            SeqLenFilter(min_seq_len=conf['coles_target.train']['min_seq_len']),
            FeatureFilter(keep_feature_names=category_names),
            CategorySizeClip(target_module.seq_encoder.category_max_size),
        ),
        shuffle_files=False,
    )
    valid_dataset = ParquetDataset(
        valid_files,
        post_processing=IterableChain(
            FeatureFilter(keep_feature_names=category_names),
            CategorySizeClip(target_module.seq_encoder.category_max_size),
        ),
        shuffle_files=False,
    )
    
    train_dataset = list(train_dataset)
    valid_dataset = list(valid_dataset)
    
    train_dataset = MapSplittingDataset(
        base_dataset=train_dataset,
        splitter=split_strategy.create(**conf['coles_target.train']['split_strategy']),
        a_chain=lambda x: x,
    )
    valid_dataset = MapSplittingDataset(
        base_dataset=valid_dataset,
        splitter=split_strategy.create(**conf['coles_target.valid']['split_strategy']),
        a_chain=lambda x: x,
    )
    
    train_dataloader = DataLoader(
        dataset=train_dataset,
        collate_fn=coles_collate_fn,
        shuffle=True,
        num_workers=conf['coles_target.train']['num_workers'],
        batch_size=conf['coles_target.train']['batch_size'],
    )
    val_dataloader = DataLoader(
        dataset=valid_dataset,
        collate_fn=coles_collate_fn,
        num_workers=conf['coles_target.valid']['num_workers'],
        batch_size=conf['coles_target.valid']['batch_size'],
    )
    
    if 'logger_name' in conf:
        _trainer_params['logger'] = TensorBoardLogger(
            save_dir='lightning_logs',
            name=conf.get('logger_name'),
        )

    trainer = pl.Trainer(
        gpus=1,
        auto_select_gpus=False,
        checkpoint_callback=False,
        logger=pl.loggers.TensorBoardLogger(
            save_dir='lightning_logs',
            name=f'[{fold_id}]: coles_model',
        ),
        **conf['coles_target.trainer']
    )
    trainer.fit(target_module, train_dataloader=train_dataloader, val_dataloaders=val_dataloader)
    
    target_model = target_module.seq_encoder
    target_model.is_reduce_sequence = True
    
    # make inference on train for LGB
    train_dataloader = DataLoader(
        dataset=get_dataset(conf['data_module'], target_model, fold['target']),
        collate_fn=padded_collate,
        shuffle=False,
        num_workers=conf['data_module.test.num_workers'],
        batch_size=conf['data_module.test.batch_size'],
        drop_last=False,
    )
    train_X, train_y = score_model2(target_model, train_dataloader, {'device': 'cuda'})
    train_y = np.array([e['y'] for e in train_y])
    lgb_cls = lightgbm.LGBMClassifier(**conf['lgbm_params'])
    lgb_cls.fit(train_X, train_y)
    
    for fields in estimate_data(target_model, lgb_cls, files=fold['valid'],
                                pretrained_attacks=pretrained_attacks):
        yield 'coles_embeddings', 'lgb_cls', 'valid', *fields
    for fields in estimate_data(target_model, lgb_cls, files=fold['test'],
                                pretrained_attacks=pretrained_attacks):
        yield 'coles_embeddings', 'lgb_cls', 'test', *fields

In [31]:
def train_target_model_rnn(fold_id, fold, pretrained_attacks):
#     s = ListSplitter(fold['target'], valid_size=0.1, seed=42)
#     train_files, valid_files = s.train, s.valid

    target_model = SequenceToTarget(conf['target_rnn.params'])
    target_dm = AdversarialTrainDataModule(conf['data_module'], target_model, fold['target'], [])
    target_dm.prepare_data()
    trainer = pl.Trainer(
        logger=pl.loggers.TensorBoardLogger(
            save_dir='lightning_logs',
            name=f'[{fold_id}]: target_rnn',
        ),
        callbacks=[
            pl.callbacks.EarlyStopping(monitor='val_auroc', mode='max', min_delta=0.0005, patience=2)
        ],
        **conf['target_rnn.trainer']
    )
    trainer.fit(target_model, target_dm.train_dataloader())
    target_model = target_model.seq_encoder
        
    # make inference on train for LGB
    train_dataloader = DataLoader(
        dataset=get_dataset(conf['data_module'], target_model, fold['target']),
        collate_fn=padded_collate,
        shuffle=False,
        num_workers=conf['data_module.test.num_workers'],
        batch_size=conf['data_module.test.batch_size'],
        drop_last=False,
    )
    train_X, train_y = score_model2(target_model, train_dataloader, {'device': 'cuda'})
    train_y = np.array([e['y'] for e in train_y])
    lgb_cls = lightgbm.LGBMClassifier(**conf['lgbm_params'])
    lgb_cls.fit(train_X, train_y)
    
    for fields in estimate_data(target_model, lgb_cls, files=fold['valid'],
                                pretrained_attacks=pretrained_attacks):
        yield 'rnn_model', 'lgb_cls', 'valid', *fields
    for fields in estimate_data(target_model, lgb_cls, files=fold['test'],
                                pretrained_attacks=pretrained_attacks):
        yield 'rnn_model', 'lgb_cls', 'test', *fields

In [32]:
results = []
for fold_id, fold in data_splitter.folds.items():
#     if fold_id == 0:
#         continue
    
    substitute_model, mcc_freq = train_substitute_model(fold_id, fold, conf['substitute'])
    attack_dm = AdversarialAttackDataModule(
        conf['data_module'], substitute_model, mcc_freq, fold['valid'] + fold['test'])
    attack_dm.prepare_data()
    
    logger.info('Calculate attacks')
    ts, original_scores = tuple(score_model2(substitute_model, attack_dm.test_dataloader(), {'device': 'cuda'}))
    pd_scores = pd.DataFrame(original_scores.tolist())
    pd_scores['s_score'] = ts
    adv_sequences = pd_scores.groupby('id').apply(lambda x: x.sort_values('s_score').iloc[0]).set_index('id') \
        .drop(columns='y').to_dict(orient='index')
    
    pa = {k: v['new_trx'] for k, v in adv_sequences.items()}
    for fields in train_target_model_rnn(fold_id, fold, pa):
        print(f'[{fold_id}]: {fields}')
        results.append(fields)
        
#     for fields in train_target_model_coles(fold_id, fold, pa):
#         print(f'[{fold_id}]: {fields}')
#         results.append(fields)

#     for fields in train_target_model_agg_feature(fold_id, fold, pa):
#         print(f'[{fold_id}]: {fields}')
#         results.append(fields)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


0it [00:00, ?it/s]

2021-06-22 16:18:51,061 INFO    setup_map              : Loaded 2517 for train model


0it [00:00, ?it/s]

2021-06-22 16:18:51,117 INFO    setup_map              : Loaded 398 for valid model
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
2021-06-22 16:18:56,222 INFO    get_lr_scheduler       : StepLR lr_scheduler used

  | Name          | Type          | Params
------------------------------------------------
0 | loss          | BCELoss       | 0     
1 | _seq_encoder  | RnnSeqEncoder | 29.8 K
2 | _head         | Sequential    | 145   
3 | train_metrics | ModuleDict    | 0     
4 | valid_metrics | ModuleDict    | 0     
5 | test_metrics  | ModuleDict    | 0     
------------------------------------------------
29.9 K    Trainable params
0         Non-trainable params
29.9 K    Total params
0.120     Total estimated model params size (MB)


...

0it [00:00, ?it/s]    

[4]: ('rnn_model', 'lgb_cls', 'valid', 'auroc_adv', 0.8492103207226345)
[4]: ('rnn_model', 'lgb_cls', 'valid', 'recall_adv', 0.6010830324909747)
[4]: ('rnn_model', 'lgb_cls', 'valid', 'precision_adv', 0.808252427184466)
[4]: ('rnn_model', 'lgb_cls', 'valid', 'p_drop', 0.10518400734096262)


0it [00:00, ?it/s]    

[4]: ('rnn_model', 'lgb_cls', 'test', 'auroc_orig', 0.8412624342719266)
[4]: ('rnn_model', 'lgb_cls', 'test', 'recall_orig', 0.7131979695431472)
[4]: ('rnn_model', 'lgb_cls', 'test', 'precision_orig', 0.7635869565217391)


[4]: ('rnn_model', 'lgb_cls', 'test', 'auroc_adv', 0.8346497917188319)
[4]: ('rnn_model', 'lgb_cls', 'test', 'recall_adv', 0.5736040609137056)
[4]: ('rnn_model', 'lgb_cls', 'test', 'precision_adv', 0.8308823529411765)
[4]: ('rnn_model', 'lgb_cls', 'test', 'p_drop', 0.10287925271601532)


In [33]:
pd.DataFrame(results, columns=['encoder', 'target_model', 'set', 'metric', 'value']) \
    .pivot_table(index=['encoder', 'target_model', 'metric'],
                 columns='set', values='value', aggfunc=['mean', t_pm, t_int_l, t_int_h, 'count']) \
    .swaplevel(axis=1)[['valid', 'test']].round(3)

set                                    valid                               \
                                        mean   t_pm t_int_l t_int_h count   
encoder   target_model metric                                               
rnn_model lgb_cls      auroc_adv       0.831  0.020   0.811   0.851     5   
                       auroc_orig      0.835  0.019   0.816   0.854     5   
                       p_drop          0.105  0.033   0.072   0.139     5   
                       precision_adv   0.801  0.037   0.763   0.838     5   
                       precision_orig  0.733  0.020   0.713   0.754     5   
                       recall_adv      0.577  0.059   0.518   0.636     5   
                       recall_orig     0.700  0.021   0.679   0.721     5   

set                                     test                               
                                        mean   t_pm t_int_l t_int_h count  
encoder   target_model metric                                              
rnn_model lgb_cls      auroc_adv       0.842  0.009   0.834   0.851     5  
                       auroc_orig      0.847  0.007   0.840   0.854     5  
                       p_drop          0.101  0.035   0.066   0.137     5  
                       precision_adv   0.839  0.031   0.808   0.870     5  
                       precision_orig  0.772  0.014   0.758   0.785     5  
                       recall_adv      0.593  0.050   0.543   0.642     5  
                       recall_orig     0.720  0.020   0.700   0.740     5

In [ ]:
print(pd.DataFrame(results, columns=['encoder', 'target_model', 'set', 'metric', 'value']) \
    .pivot_table(index=['encoder', 'target_model', 'metric'],
                 columns='set', values='value', aggfunc=['mean', t_pm, t_int_l, t_int_h, 'count']) \
    .swaplevel(axis=1)[['valid', 'test']].round(3).to_markdown())

|                                        | valid |       |         |         |       | test  |       |         |         |       |
|:---------------------------------------|------:|------:|--------:|--------:|------:|------:|------:|--------:|--------:|------:|
|                                      | **mean**  | **t_pm**  | **t_int_l** | **t_int_h** | **count** | **mean**  | **t_pm**  | **t_int_l** | **t_int_h** | **count** |
| agg_features, lgb_cls, auroc_adv       | 0.857 | 0.013 |   0.844 |   0.87  |     5 | 0.86  | 0.006 |   0.855 |   0.866 |     5 |
| agg_features, lgb_cls, auroc_orig      | 0.866 | 0.014 |   0.852 |   0.879 |     5 | 0.869 | 0.004 |   0.865 |   0.874 |     5 |
| agg_features, lgb_cls, p_drop          | 0.085 | 0.014 |   0.071 |   0.1   |     5 | 0.083 | 0.015 |   0.068 |   0.099 |     5 |
| agg_features, lgb_cls, recall_adv      | 0.548 | 0.043 |   0.505 |   0.591 |     5 | 0.565 | 0.021 |   0.544 |   0.586 |     5 |
| agg_features, lgb_cls, recall_orig     | 0.702 | 0.036 |   0.666 |   0.738 |     5 | 0.708 | 0.013 |   0.695 |   0.721 |     5 |
| coles_embeddings, lgb_cls, auroc_adv   | 0.841 | 0.016 |   0.825 |   0.857 |     5 | 0.846 | 0.009 |   0.838 |   0.855 |     5 |
| coles_embeddings, lgb_cls, auroc_orig  | 0.855 | 0.016 |   0.838 |   0.871 |     5 | 0.86  | 0.01  |   0.85  |   0.87  |     5 |
| coles_embeddings, lgb_cls, p_drop      | 0.098 | 0.026 |   0.073 |   0.124 |     5 | 0.097 | 0.03  |   0.068 |   0.127 |     5 |
| coles_embeddings, lgb_cls, recall_adv  | 0.544 | 0.051 |   0.493 |   0.595 |     5 | 0.543 | 0.047 |   0.497 |   0.59  |     5 |
| coles_embeddings, lgb_cls, recall_orig | 0.708 | 0.022 |   0.686 |   0.73  |     5 | 0.703 | 0.03  |   0.673 |   0.733 |     5 |
| rnn_model, lgb_cls, auroc_adv          | 0.831 | 0.02  |   0.811 |   0.851 |     5 | 0.842 | 0.009 |   0.834 |   0.851 |     5 |
| rnn_model, lgb_cls, auroc_orig         | 0.835 | 0.019 |   0.816 |   0.854 |     5 | 0.847 | 0.007 |   0.84  |   0.854 |     5 |
| rnn_model, lgb_cls, p_drop             | 0.105 | 0.033 |   0.072 |   0.139 |     5 | 0.101 | 0.035 |   0.066 |   0.137 |     5 |
| rnn_model, lgb_cls, precision_adv      | 0.801 | 0.037 |   0.763 |   0.838 |     5 | 0.839 | 0.031 |   0.808 |   0.87  |     5 |
| rnn_model, lgb_cls, precision_orig     | 0.733 | 0.02  |   0.713 |   0.754 |     5 | 0.772 | 0.014 |   0.758 |   0.785 |     5 |
| rnn_model, lgb_cls, recall_adv         | 0.577 | 0.059 |   0.518 |   0.636 |     5 | 0.593 | 0.05  |   0.543 |   0.642 |     5 |
| rnn_model, lgb_cls, recall_orig        | 0.700 | 0.021 |   0.679 |   0.721 |     5 | 0.72  | 0.02  |   0.7   |   0.74  |     5 |
